In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from IPython.core.display import display, HTML
from cross_validation import *
from build_polynomial import *
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))


## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
#y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
#print(y.shape, tX.shape, ids.shape)

In [ ]:
import pandas as pd
df = pd.read_csv("../data/train.csv")
df.tail()

In [ ]:
y0, tX0, id0 = load_data_jet_number(DATA_TRAIN_PATH, 0, sub_sample=True)
y1, tX1, id1 = load_data_jet_number(DATA_TRAIN_PATH, 1, sub_sample=True)
y2, tX2, id2 = load_data_jet_number(DATA_TRAIN_PATH, 2, sub_sample=True)
y3, tX3, id3 = load_data_jet_number(DATA_TRAIN_PATH, 3, sub_sample=True)
tX0_standardized, tX0_mean, tX0_std= standardize(tX0)
tX1_standardized, tX1_mean, tX1_std= standardize(tX1)
tX2_standardized, tX2_mean, tX2_std= standardize(tX2)
tX3_standardized, tX3_mean, tX3_std= standardize(tX3)

In [ ]:
print(y0.shape, tX0.shape, id0.shape)

In [ ]:
print(np.std(tX0_standardized))

## Do your thing crazy machine learning thing here :) ...

In [ ]:
tX0_standardized = np.c_[np.ones((y0.shape[0], 1)), tX0_standardized] #VERY IMPORTANT
y0=y0.reshape(-1,1)
y0=(y0+1)/2
tX1_standardized = np.c_[np.ones((y1.shape[0], 1)), tX1_standardized] #VERY IMPORTANT
y1=y1.reshape(-1,1)
y1=(y1+1)/2
tX2_standardized = np.c_[np.ones((y2.shape[0], 1)), tX2_standardized] #VERY IMPORTANT
y2=y2.reshape(-1,1)
y2=(y2+1)/2
tX3_standardized = np.c_[np.ones((y3.shape[0], 1)), tX3_standardized] #VERY IMPORTANT
y3=y3.reshape(-1,1)
y3=(y3+1)/2

In [ ]:
print(tX0_standardized.shape)
print(np.mean(tX0_standardized))

In [ ]:
[w,loss]=reg_logistic_regression(y0, tX0_standardized, 0.001,np.ones((tX0_standardized.shape[1], 1)), 10000, 0.0001)

In [ ]:
#lambda_=0.0000001
k_fold=4
seed=5
degree=5 #will -1 as range exclude last elem
lower_lambda=-8
upper_lambda=0

weights_0, loss_0, deg0 = cross_validation_best_weight(y0, tX0_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet0")
weights_1, loss_1, deg1 = cross_validation_best_weight(y1, tX1_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet1")
weights_2, loss_2, deg2 = cross_validation_best_weight(y2, tX2_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet2")
weights_3, loss_3, deg3 = cross_validation_best_weight(y3, tX3_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet3")

# Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
print(tX_test.shape, ids_test.shape)

In [ ]:
_, tX0_te, id0_te = load_data_jet_number("../data/test.csv", 0)
_, tX1_te, id1_te = load_data_jet_number("../data/test.csv", 1)
_, tX2_te, id2_te = load_data_jet_number("../data/test.csv", 2)
_, tX3_te, id3_te = load_data_jet_number("../data/test.csv", 3)
tX0_te_standardized, tX0_te_mean, tX0_te_std= standardize(tX0_te)
tX1_te_standardized, tX1_te_mean, tX1_te_std= standardize(tX1_te)
tX2_te_standardized, tX2_te_mean, tX2_te_std= standardize(tX2_te)
tX3_te_standardized, tX3_te_mean, tX3_te_std= standardize(tX3_te)

In [ ]:
y0_pred = predict_labels(w, a)

In [ ]:
print(y0_pred[0:100])

In [ ]:
OUTPUT_PATH = '../data/pred.csv' # TODO: fill in desired name of output file for submission
y0_pred = predict_labels(weights_0, build_poly(tX0_te_standardized,deg0))
y1_pred = predict_labels(weights_1, build_poly(tX1_te_standardized,deg1))
y2_pred = predict_labels(weights_2, build_poly(tX2_te_standardized,deg2))
y3_pred = predict_labels(weights_3, build_poly(tX3_te_standardized,deg3))

In [ ]:
y_pred_all=np.concatenate([y0_pred,y1_pred,y2_pred,y3_pred])
ids_pred_all=np.concatenate([id0_te,id1_te,id2_te,id3_te])
y_and_ids=list(zip(*sorted(zip(ids_pred_all, y_pred_all))))
ids_test=y_and_ids[0]
y_pred=y_and_ids[1]

In [ ]:
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)